In [1]:
#This versions creates just cbg nodes. 
import pandas as pd #importing neccisary libraries 
import numpy as np
import ast
import datetime
import calendar

In [124]:
# you might have to change the uri to whatever pops up when you run neo
#connect to neo4j database 
from neo4j import GraphDatabase 
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password")) 
session = driver.session()

In [2]:
csvs =['feb2020.csv', 'april2020.csv', 'july2020.csv','june2021.csv']

In [135]:
for i in range(len(csvs)):
    
    df = pd.read_csv(csvs[i]) # read csv to pandas

    #isolate specified colums will be changing them to more meanningful names

    miamivv = df.visitor_home_cbgs
    # Will add another part to have all the csv be read at once so I do not have to keep loading it.
    #This for loop creates and loads all nodes and edges

    #len(df)
    #this for loop I am createing all the places and cities and zips
    for i in range(len(df)):
        #setting variables that I will use to create the nodes. 
        pk= df.placekey[i]
        name = df.location_name[i]
        zipc= df.postal_code[i]
        address= df.street_address[i]
        lat= df.latitude[i]
        long= df.longitude[i]
        topCat= df.top_category[i]
        subCat= df.sub_category[i]
        city= df.city[i]
        if((str(df.poi_cbg[i]) == 'nan') == False):
            poicbg = str(df.poi_cbg[i])
            poiST= str(poicbg[:2])
            poiCTY= str(poicbg[2:5])
            check = 0 # This is telling the program that this row has the information we need
            
        
        else:        
            check = 1 # if not we skip over this row
                  
    
        if (check == 0):
            #These are the queries that I will pass to neo4j that will create all the place, city and zip nodes    
            q1='MERGE(p:Place{name:'+'"'+name+'"' +', city:'+'"' +city+'"'+', zipcode:'+"'" +str(zipc)+"'" +', placeKey:'+"'" +pk+"'" +', address:'+'"' +address+'"' +', topCatagory:'+'"'+str(topCat)+'"'+', subCatagory:'+'"'+str(subCat)+'"' +', cbg: '+"'" +str(poicbg) +"'" +', location:point({x:toFloat('+str(lat)+'),y:toFloat('+str(long)+')})})'
            q2='MERGE(z:Zip{name: '+"'" +str(zipc)+"'" +', zipC:'+"'"+str(zipc)+"'"+', city:'+"'" +city+"'"+'});'
            q3='MERGE(c:City{name: '+'"' +city+'"'+',  city:'+"'" +city+"'"+'});'
            q4='MATCH (p:Place), (z:Zip) WHERE p.name = '+'"' +name +'"' + 'AND p.city = '+"'" + city +"'"+ 'AND z.city = '+"'" +city+"'" + 'AND p.zipcode = '+"'" +str(zipc)+"'" +' AND z.name = '+"'" +str(zipc)+"'" +' MERGE(p)-[r:IS_WITHIN]->(z);'
            q5='MATCH (c:City), (z:Zip) WHERE c.name = '+'"' +city +'"' +' AND z.city = '+"'" +city+"'" + ' AND z.name = '+"'" +str(zipc)+"'" +' MERGE(z)-[r:IS_WITHIN]->(c);'             
            q6='MERGE(g:CensusBG{cbg:'+"'" +str(poicbg)+"'"+ ', State:'+"'"+str(poiST)+"'"+' , County:'+"'"+str(poiCTY)+"'"+' })'
            q7='MATCH (p:Place), (g:CensusBG) WHERE p.placeKey = '+'"' +pk+'"' +' AND g.State = '+"'" +str(poiST)+"'" +' AND g.cbg = '+"'" +str(poicbg)+"'" + ' MERGE(p)-[r:IS_WITHIN]->(g);'
            #Match staments are the edges that connect the two nodes
    
            #running the querries 
            session.run(q1)
            session.run(q2)
            session.run(q3)
            session.run(q4)
            session.run(q5)
            session.run(q6)
            session.run(q7)
    
            # miamivv =  df.visitor_home_cbgs
            #json ->dictionary-> nparray
            if((type(miamivv[i]) == float) == True):
                iti= 0

            else:
                test=ast.literal_eval(miamivv[i])
                test2 = list(test.items())
                testnpp = np.array(test2)
                iti = len(testnpp)


            if iti != 0:

                da=0
                dasum=0
                percentage = []
                rawVists = df.raw_visit_counts[i]
                dates= df.date_range_start[i]
                rawVisitors = df.raw_visitor_counts[i]
        
                #parsing the date month and year
                year=dates[:4]
                month=dates[5:7]
                monthName=calendar.month_name[int(month)]
        
                for x in range(iti):

                    da = int(testnpp[x][1])   
                    dasum += da # sum 
                    percentage =[]
                   
        

                for j in range(len(testnpp)):
                    #here I am createing Census block nodes and connecting them with the places they have been. 
                    odds = (float((testnpp[j][1]))/dasum)  
                    percentage += [odds]
                    p1= float(percentage[j] *100)
                    evc= float(((p1/100) * rawVists))
                    NoCBGVisitors = float(100*((rawVisitors-dasum)/rawVisitors))            
                    node= testnpp[j][0] # this is the keys
                        
                    if(node[0].isupper() == True):
                        nodee =(node)
                        state = node[:2]
                        county = node[3:]
                
            
                    else:                
                        nodee = float(node)
                        prop= testnpp[j][1] # this is the values         
                        state = node[:2]
                        county = node[2:5]
            
                    qq1= 'MERGE(g:CensusBG{cbg:'+"'" +str(nodee)+"'"+ ', State:'+"'"+state+"'"+' , County:'+"'"+county+"'"+' })'
                    qq2 ='MATCH (p:Place), (g:CensusBG) WHERE p.placeKey = '+"'" +pk +"'"  +' AND g.cbg = '+"'" +str(nodee)+"'" +' MERGE(g)-[b:Visited{ PercentofVisits:'+str(p1)+', EstimatedVisits:'+str(evc)+', NoCbgPercentage:'+str(NoCBGVisitors)+', month:'+"'"+str(monthName)+"'"+', year:'+str(year)+'}]->(p)'
            
                    session.run(qq1) 
                    session.run(qq2)

In [3]:
#Also if you stop the kernel you will have to re run the parts where you connect to neo
#a short cut will be just running the secound cell again